In [1]:
async def get_player_gameweek_data(player_id, session):
    """Fetch gameweek data and upcoming fixtures for a specific player."""
    url = f"https://fantasy.premierleague.com/api/element-summary/{player_id}/"
    async with session.get(url) as response:
        data = await response.json()
        gameweek_data = data['history']  # Gameweek data for this player
        fixtures = data['fixtures']  # Upcoming fixtures for this player
        return gameweek_data, fixtures

In [2]:
import aiohttp
import pandas as pd
import numpy as np
from datetime import date

url = "https://fantasy.premierleague.com/api/bootstrap-static/"

async with aiohttp.ClientSession() as session:
    async with session.get(url) as response:
        data = await response.json()
        players = data['elements']  # List of all players
        teams = data['teams']  # List of teams

        # Create a mapping from team ID to team name
        team_mapping = {team['id']: team['name'] for team in teams}

        # Extract a comprehensive set of fields for each player
        players_data = []
        for player in players:
            # Fetch gameweek data and fixtures for this player
            gameweek_data, fixtures = await get_player_gameweek_data(player['id'], session)

            # Calculate average fixture difficulty for upcoming matches
            upcoming_fixtures = [f for f in fixtures if not f['finished']]
            if upcoming_fixtures:
                avg_difficulty = np.mean([f['difficulty'] for f in upcoming_fixtures])
            else:
                avg_difficulty = np.nan  # No upcoming fixtures

            player_info = {
                "Player ID": player['id'],
                "Player Name": player['web_name'],
                "Full Name": f"{player['first_name']} {player['second_name']}",
                "Team ID": player['team'],
                "Team Name": team_mapping[player['team']],
                "Position": player['element_type'],
                "Cost (£)": player['now_cost'] / 10,
                "Total Points": player['total_points'],
                "ICT Index": player['ict_index'],
                "Threat": player['threat'],  # Most recent threat score
                "Points Per Game": player['points_per_game'],
                "Expected Points (This GW)": player['ep_this'],
                "Expected Points (Next GW)": player['ep_next'],
                "Current Form": player['form'],
                "Status": player['status'],
                "Chance of Playing Next GW (%)": player.get('chance_of_playing_next_round', "N/A"),
                "News": player['news'],
                "Avg Fixture Difficulty (Upcoming)": avg_difficulty
            }

            # Append each gameweek's data as a row, adding player details
            for gw in gameweek_data:
                gameweek_info = {
                    **player_info,
                    "Game Week": gw['round'],
                    "Goals Scored": gw['goals_scored'],
                    "Assists": gw['assists'],
                    "Clean Sheets": gw['clean_sheets'],
                    "Minutes Played": gw['minutes'],
                    "Bonus Points": gw['bonus'],
                    "Yellow Cards": gw['yellow_cards'],
                    "Red Cards": gw['red_cards'],
                    "Points": gw['total_points'],
                    "Transfers In": gw['transfers_in'],
                    "Transfers Out": gw['transfers_out']
                }
                players_data.append(gameweek_info)

        # Create a DataFrame with player stats by gameweek
        df = pd.DataFrame(players_data)

        #df = df.sort_values('Game Week', ascending=False).drop_duplicates('Player ID')

        # Calculate exponential weights for each gameweek
        alpha = 0.1  # Growth rate for weights
        df['Weight'] = np.exp(alpha * (df['Game Week'] - 1))
        df['Weighted Points'] = df['Points'] * df['Weight']

        # Calculate over and under 3 points stats
        df['Over 3 Points'] = df['Points'] > 3
        df['3 Points and Under'] = df['Points'] <= 3
        
        #rename columns for big query compliance
        df.rename(columns={
            "Player ID": 'player_id',
            'Full Name': 'full_name',
            'Player Name': 'player_name',
            'Team Name': 'team_name',
            'Position': 'position',
            'Cost (£)': 'cost',
            'Total Points': 'total_points',
            'Weighted Points': 'weighted_points',
            'Recent 4 Weeks Points': 'recent_4_weeks_points',
            'Recent 8 Weeks Points': 'recent_8_weeks_points',
            'Expected Points (This GW)': 'expected_points_this_gw',
            'Expected Points (Next GW)': 'expected_points_next_gw',
            'ICT Index': 'ict_index',
            'Threat': 'threat',
            'Points Per Game': 'points_per_game',
            'Current Form': 'current_form',
            'Status': 'status',
            'Chance of Playing Next GW (%)': 'chance_of_playing_next_gw_percent',
            'News': 'news',
            'Avg Fixture Difficulty (Upcoming)': 'avg_fixture_difficulty_upcoming',
            'Game Week': 'game_week',
            'Goals Scored': 'goals_scored',
            'Assists': 'assists',
            'Clean Sheets': 'clean_sheets',
            'Minutes Played': 'minutes_played',
            'Bonus Points': 'bonus_points',
            'Yellow Cards': 'yellow_cards',
            'Red Cards': 'red_cards',
            'Transfers In': 'transfers_in',
            'Transfers Out': 'transfers_out',
            'Over 3 Points': 'over_3_points',
            '3 Points and Under': 'three_points_and_under',
        }, inplace=True)

In [5]:
df.to_excel(f"data/Weekly_Player_Data.xlsx", index=False)

In [7]:
df_agg=df
Recent_weeks_data = pd.DataFrame()

#Weekly aggregations
for week in df_agg['game_week'].unique():
    # Calculate sum of total points for the last 4 gameweeks
    LB_week = week - 3
    recent_weeks = [i for i in range(LB_week, week + 1) if i>= 1]
    week_range=df_agg[df_agg['game_week'].isin(recent_weeks)]
    week_range_grouped = week_range.groupby('player_id').agg({'Points': 'sum'}).reset_index()

    # Calculate sum of total points for the last 4 gameweeks
    LB_week = week - 7
    recent_weeks_8 = [i for i in range(LB_week, week + 1) if i>= 1]
    week_range_8=df_agg[df_agg['game_week'].isin(recent_weeks_8)]
    week_range_grouped_8 = week_range_8.groupby('player_id').agg({'Points': 'sum'}).reset_index()

    # Merge the weekly aggregations
    week_range_merged = week_range_grouped.merge(
        week_range_grouped_8,
        on='player_id', how='left'
    )

    # Add the current week to the grouped data
    week_range_merged['game_week'] = week
    
    # Rename the columns for clarity
    week_range_merged.rename(columns={'Points_x': 'recent_4_weeks_points', 'Points_y': 'recent_8_weeks_points'}, inplace=True)
    
    #-------------------------------------------------------------------------------------------------
    
    #total metrics for player IDs
    df_metric_total = df_agg[df_agg['game_week'].isin(range(1,week+1))]

    #measure if a game was played by player ID
    df_metric_total['games_played'] = [1 if i > 0 else 0 for i in df_metric_total['Points']]

    #aggregate metrics for the week by player ID
    df_metric_total = df_metric_total.groupby('player_id').agg({
        'total_points': 'sum',
        'goals_scored': 'sum',
        'assists': 'sum',
        'clean_sheets': 'sum',
        'minutes_played': 'sum',
        'bonus_points': 'sum',
        'yellow_cards': 'sum',
        'red_cards': 'sum',
        'transfers_in': 'sum',
        'transfers_out': 'sum',
        'games_played': 'sum',
        'over_3_points': 'sum',
        'three_points_and_under': 'sum'
    }).reset_index()

    # Calculate the consistency ratio
    Ratio = df_metric_total['over_3_points'] / df_metric_total['three_points_and_under']
    df_metric_total['Ratio'] = Ratio * df_metric_total['games_played']

    # Merge the weekly aggregations with the total metrics
    week_range_metric_merged = week_range_merged.merge(
        df_metric_total,
        on='player_id', how='left'
    )

    # Append the weekly data to the main DataFrame
    Recent_weeks_data=pd.concat([Recent_weeks_data,week_range_metric_merged], ignore_index=True)

Recent_weeks_data.to_excel(f"data/aggregated_weekly_player_data.xlsx", index=False)

C:\Users\arabu\AppData\Local\Temp\ipykernel_34132\1059824519.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_metric_total['games_played'] = [1 if i > 0 else 0 for i in df_metric_total['Points']]
C:\Users\arabu\AppData\Local\Temp\ipykernel_34132\1059824519.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_metric_total['games_played'] = [1 if i > 0 else 0 for i in df_metric_total['Points']]
C:\Users\arabu\AppData\Local\Temp\ipykernel_34132\1059824519.py:36: SettingWithCopyWarning: 
A value is tr

In [ ]:
Recent_weeks_data

,Player ID,Recent 4 Weeks Points,Recent 8 Weeks Points,Game Week,Points,Goals Scored,Assists,Clean Sheets,Minutes Played,Bonus Points,Yellow Cards,Red Cards,Transfers In,Transfers Out,Games Played,Over 3 Points,3 Points and Under,Ratio
0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0
1,2,0,0,1,0,0,0,0,5,0,1,0,0,0,0,0,1,0.0
2,3,6,6,1,6,0,0,1,90,0,0,0,0,0,1,1,0,inf
3,4,12,12,1,12,1,1,1,90,3,0,0,0,0,1,1,0,inf
4,5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5203,663,0,0,8,0,0,0,0,0,0,0,0,1491,278,0,0,3,0.0
5204,664,0,0,8,0,0,0,0,0,0,0,0,13110,3312,0,0,3,0.0
5205,665,0,0,8,0,0,0,0,0,0,0,0,8358,1102,0,0,2,0.0
5206,666,0,0,8,0,0,0,0,0,0,0,0,388,59,0,0,2,0.0


In [ ]:
Recent_weeks_data[Recent_weeks_data['Player ID']== 328]

,Player ID,Recent 4 Weeks Points,Recent 8 Weeks Points,Game Week,Points,Goals Scored,Assists,Clean Sheets,Minutes Played,Bonus Points,Yellow Cards,Red Cards,Transfers In,Transfers Out,Games Played,Over 3 Points,3 Points and Under,Ratio
327,328,14,14,1,14,1,1,1,90,3,0,0,0,0,1,1,0,inf
943,328,24,24,2,24,2,1,2,172,5,0,0,262302,64190,2,2,0,inf
1570,328,41,41,3,41,3,3,3,262,8,0,0,570828,250317,3,3,0,inf
2218,328,43,43,4,43,3,3,3,352,8,0,0,1470182,317307,4,3,1,12.0
2877,328,35,49,5,49,3,4,4,442,8,0,0,1706995,392081,5,4,1,20.0
3538,328,35,59,6,59,4,4,4,532,11,0,0,1805024,922528,6,5,1,30.0
4202,328,21,62,7,62,4,4,5,604,11,0,0,1886240,1274492,7,5,2,17.5
4868,328,31,74,8,74,5,5,5,694,13,0,0,1942105,1582746,8,6,2,24.0


In [ ]:
Recent_weeks_data[Recent_weeks_data['Full Name']== 'Mohamed Salah'][['Game Week','Player Name','Recent 4 Weeks Points','Recent 8 Weeks Points']]

,Game Week,Player Name,Recent 4 Weeks Points,Recent 8 Weeks Points
15961,1,M.Salah,0,0
15962,2,M.Salah,0,0
15963,3,M.Salah,0,0
15964,4,M.Salah,0,0
15965,5,M.Salah,0,0
15966,6,M.Salah,0,0
15967,7,M.Salah,0,0
15968,8,M.Salah,0,0
15969,9,M.Salah,0,0
15970,10,M.Salah,0,0
